In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
from IPython.display import display, HTML
init_notebook_mode(connected=True)
from os import listdir
datadir = '/home/jovyan/work/dswg/data'
print(listdir(datadir))

['medications.csv', 'temp', 'allergies.csv', 'procedures.csv', 'conditions.csv', 'careplans.csv', 'encounters.csv', 'immunizations.csv', '~$AllPrescribableOpioidsUsedForPainControlIncludingInactiveMedications.xlsx', 'AllPrescribableOpioidsUsedForPainControlIncludingInactiveMedications.xlsx', 'patients.csv', 'observations.csv']


In [28]:
# Read in the data
dd = {}

dd['patients'] = {'pat_id':     {'type': np.str, 'required':True, 'primarykey_col':0},
                  'birth_date': {'type': np.datetime64, 'required':True},
                  'death_date': {'type': np.datetime64}, 
                  'ssn':        {'type': np.str},
                  'drivers':    {'type': np.str},
                  'passport':   {'type': np.str},
                  'prefix':     {'type': np.str},
                  'first':      {'type': np.str, 'required':True},
                  'last':       {'type': np.str, 'required':True},
                  'suffix':     {'type': np.str},
                  'maiden':     {'type': np.str},
                  'marital':    {'type': np.str},
                  'race':       {'type': np.str},
                  'ethnicity':  {'type': np.str},
                  'gender':     {'type': np.str, 'required':True},
                  'birthplace': {'type': np.str},
                  'address':    {'type': np.str, 'required':True}
                  }
dd['encounters'] = {'enc_id':                 {'type': np.str, 'required':True, 'primarykey_col':0},
                    'enc_date':               {'type': np.datetime64, 'required':True},
                    'enc_pat_id':             {'type': np.str, 'required':True},
                    'enc_code':               {'type': np.str, 'required':True},
                    'enc_description':        {'type': np.str, 'required':True},
                    'enc_reason_code':        {'type': np.str},
                    'enc_reason_description': {'type': np.str}
                   }
dd['observations'] = {'obs_date':        {'type': np.datetime64, 'required':True},
                      'obs_pat_id':      {'type': np.str, 'required':True},
                      'obs_enc_id':      {'type': np.str, 'required':True},
                      'obs_code':        {'type': np.str, 'required':True},
                      'obs_description': {'type': np.str, 'required':True},
                      'obs_value':       {'type': np.str},
                      'obs_units':       {'type': np.str}
                     }
dd['medications'] = {'med_start_date':         {'type': np.datetime64, 'required':True},
                     'med_stop_date':          {'type': np.str, 'required':False},
                     'med_pat_id':             {'type': np.str, 'required':True},
                     'med_enc_id':             {'type': np.str, 'required':True},
                     'med_code':               {'type': np.str, 'required':True},
                     'med_description':        {'type': np.str, 'required':True},
                     'med_reason_code':        {'type': np.str},
                     'med_reason_description': {'type': np.str}
                     }


data = {}
for f in dd:
    m = dd[f]
    col_names = list(m.keys())
    data_types = {k: (v['type'] if v['type'] != np.datetime64 else np.str) for (k,v) in m.items()}
    date_cols = [k for k,v in m.items() if v['type'] == np.datetime64]
    key_cols = [v['primarykey_col'] for k,v in m.items() if 'primarykey_col' in v]
    if len(key_cols) == 0:
        key_cols = None
    print('{}:'.format(f))
    print('  keys:  {}'.format(key_cols))
    print('  types: {}'.format(data_types))
    print('  cols:  {}'.format(col_names))
    print('  dates: {}'.format(date_cols))
    data[f] = pd.read_csv(datadir + '/{}.csv'.format(f), index_col=key_cols, dtype=data_types, header=0, parse_dates=date_cols, names=col_names)
    display(data[f].head(5))
    print(data[f].dtypes)

patients:
  keys:  [0]
  types: {'pat_id': <class 'str'>, 'birth_date': <class 'str'>, 'death_date': <class 'str'>, 'ssn': <class 'str'>, 'drivers': <class 'str'>, 'passport': <class 'str'>, 'prefix': <class 'str'>, 'first': <class 'str'>, 'last': <class 'str'>, 'suffix': <class 'str'>, 'maiden': <class 'str'>, 'marital': <class 'str'>, 'race': <class 'str'>, 'ethnicity': <class 'str'>, 'gender': <class 'str'>, 'birthplace': <class 'str'>, 'address': <class 'str'>}
  cols:  ['pat_id', 'birth_date', 'death_date', 'ssn', 'drivers', 'passport', 'prefix', 'first', 'last', 'suffix', 'maiden', 'marital', 'race', 'ethnicity', 'gender', 'birthplace', 'address']
  dates: ['birth_date', 'death_date']


,birth_date,death_date,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,birthplace,address
pat_id,,,,,,,,,,,,,,,,
545be6f8-8796-496a-ac87-9919542f724e,2004-02-18,NaT,999-22-9579,NaN,NaN,NaN,Noble66,Hintz995,NaN,NaN,NaN,white,french,M,Dell Groveship,740 Schmidt Heights Apt 24 Apple Valley Minnes...
7e354bf5-99c6-413d-b37c-b779dd103583,1996-05-28,NaT,999-93-9114,S99976555,X29508953X,Mr.,Colin861,Barrows492,NaN,NaN,NaN,white,polish,M,St. Louis Park,724 Anderson Route Apt 49 Dovership Minnesota US
f86ec82d-004a-46df-8cdb-4226abd6886d,2014-11-07,NaT,999-84-5231,NaN,NaN,NaN,Ashley34,Becker968,NaN,NaN,NaN,white,german,F,Huntership,775 Prosacco Ville Minneapolis Minnesota 55401 US
556e6a09-12c0-40fc-8f68-9989df047485,1959-10-29,2004-10-21,999-79-5940,S99947417,X40810024X,Mr.,Randolph418,Runolfsson901,NaN,NaN,M,white,greek,M,St. Cloud,577 Prohaska Drive Unit 73 Lino Lakes Minnesot...
58ec9442-891c-4855-9008-3e8d700eb5b8,2004-06-03,NaT,999-48-8276,NaN,NaN,NaN,Lili474,Johns824,NaN,NaN,NaN,white,german,F,Bloomington,303 Mertz Esplanade Edina Minnesota 55343 US


birth_date    datetime64[ns]
death_date    datetime64[ns]
ssn                   object
drivers               object
passport              object
prefix                object
first                 object
last                  object
suffix                object
maiden                object
marital               object
race                  object
ethnicity             object
gender                object
birthplace            object
address               object
dtype: object
encounters:
  keys:  [0]
  types: {'enc_id': <class 'str'>, 'enc_date': <class 'str'>, 'enc_pat_id': <class 'str'>, 'enc_code': <class 'str'>, 'enc_description': <class 'str'>, 'enc_reason_code': <class 'str'>, 'enc_reason_description': <class 'str'>}
  cols:  ['enc_id', 'enc_date', 'enc_pat_id', 'enc_code', 'enc_description', 'enc_reason_code', 'enc_reason_description']
  dates: ['enc_date']


,enc_date,enc_pat_id,enc_code,enc_description,enc_reason_code,enc_reason_description
enc_id,,,,,,
87216f21-b849-490a-a025-e2df2f042709,2009-01-28,545be6f8-8796-496a-ac87-9919542f724e,185349003,Encounter for check up (procedure),NaN,NaN
20081f28-9818-42f9-ab2e-903287bda3f5,1994-10-27,556e6a09-12c0-40fc-8f68-9989df047485,185349003,Encounter for check up (procedure),NaN,NaN
dc9874cc-7b49-4a48-8878-4be32b0b372a,2014-11-07,f86ec82d-004a-46df-8cdb-4226abd6886d,185349003,Encounter for check up (procedure),NaN,NaN
10b30d46-89f2-4029-98af-e11cc3cae62d,2008-06-17,7e354bf5-99c6-413d-b37c-b779dd103583,185349003,Encounter for check up (procedure),NaN,NaN
6c0896d3-2cdd-4d04-964d-0f8e6359d101,2010-02-03,545be6f8-8796-496a-ac87-9919542f724e,185349003,Encounter for check up (procedure),NaN,NaN


enc_date                  datetime64[ns]
enc_pat_id                        object
enc_code                          object
enc_description                   object
enc_reason_code                   object
enc_reason_description            object
dtype: object
observations:
  keys:  None
  types: {'obs_date': <class 'str'>, 'obs_pat_id': <class 'str'>, 'obs_enc_id': <class 'str'>, 'obs_code': <class 'str'>, 'obs_description': <class 'str'>, 'obs_value': <class 'str'>, 'obs_units': <class 'str'>}
  cols:  ['obs_date', 'obs_pat_id', 'obs_enc_id', 'obs_code', 'obs_description', 'obs_value', 'obs_units']
  dates: ['obs_date']


,obs_date,obs_pat_id,obs_enc_id,obs_code,obs_description,obs_value,obs_units
0,2009-01-28,545be6f8-8796-496a-ac87-9919542f724e,87216f21-b849-490a-a025-e2df2f042709,8302-2,Body Height,105.9,cm
1,2009-01-28,545be6f8-8796-496a-ac87-9919542f724e,87216f21-b849-490a-a025-e2df2f042709,29463-7,Body Weight,22.1,kg
2,2009-01-28,545be6f8-8796-496a-ac87-9919542f724e,87216f21-b849-490a-a025-e2df2f042709,39156-5,Body Mass Index,19.7,kg/m2
3,2014-11-07,f86ec82d-004a-46df-8cdb-4226abd6886d,dc9874cc-7b49-4a48-8878-4be32b0b372a,8302-2,Body Height,48.9,cm
4,2009-01-28,545be6f8-8796-496a-ac87-9919542f724e,87216f21-b849-490a-a025-e2df2f042709,8462-4,Diastolic Blood Pressure,81.5,mmHg


obs_date           datetime64[ns]
obs_pat_id                 object
obs_enc_id                 object
obs_code                   object
obs_description            object
obs_value                  object
obs_units                  object
dtype: object
medications:
  keys:  None
  types: {'med_start_date': <class 'str'>, 'med_stop_date': <class 'str'>, 'med_pat_id': <class 'str'>, 'med_enc_id': <class 'str'>, 'med_code': <class 'str'>, 'med_description': <class 'str'>, 'med_reason_code': <class 'str'>, 'med_reason_description': <class 'str'>}
  cols:  ['med_start_date', 'med_stop_date', 'med_pat_id', 'med_enc_id', 'med_code', 'med_description', 'med_reason_code', 'med_reason_description']
  dates: ['med_start_date']


,med_start_date,med_stop_date,med_pat_id,med_enc_id,med_code,med_description,med_reason_code,med_reason_description
0,2011-03-13,2011-06-11,545be6f8-8796-496a-ac87-9919542f724e,7ab0d43c-9dd3-4a42-ace9-343d73298b42,282464,Acetaminophen 160 MG Oral Tablet,NaN,NaN
1,2011-02-11,2011-02-24,9e96975e-60fa-46a6-9a51-c2f771ae664e,2d21a0b1-2b1e-4ae3-bebf-2ef5ed30db9b,834060,Penicillin V Potassium 250 MG,43878008,Streptococcal sore throat (disorder)
2,2012-10-20,2012-10-27,58ec9442-891c-4855-9008-3e8d700eb5b8,bbe739df-42e2-4dd7-baf2-e06d9bf66ecd,608680,Acetaminophen 160 MG,10509002,Acute bronchitis (disorder)
3,2010-11-11,2010-11-18,7e354bf5-99c6-413d-b37c-b779dd103583,420ab1fe-9810-4b0a-b2f4-caffdbc6be31,1020137,Dextromethorphan Hydrobromide 1 MG/ML,10509002,Acute bronchitis (disorder)
4,2015-06-11,2015-07-02,58ec9442-891c-4855-9008-3e8d700eb5b8,6a44ffe4-14d5-41b1-aebd-22e46426be51,824184,Amoxicillin 250 MG / Clavulanate 125 MG [Augme...,444814009,Viral sinusitis (disorder)


med_start_date            datetime64[ns]
med_stop_date                     object
med_pat_id                        object
med_enc_id                        object
med_code                          object
med_description                   object
med_reason_code                   object
med_reason_description            object
dtype: object


In [29]:
patients = data['patients']
encounters = data['encounters']
observations = data['observations']
medications = data['medications']

In [ ]:
# Change the column names


In [ ]:
# Inject data quality errors

In [38]:
# Add Days Supply information (bias toward more for Drug Overdose patients)
# Find all Overdose Patients
overdose_patients = set(encounters[encounters['enc_reason_code']=='55680006']['enc_pat_id'])
print(len(overdose_patients))
display(encounters[encounters['enc_pat_id'].isin(overdose_patients)].head(5))
overdose_meds = medications[medications['med_pat_id'].isin(overdose_patients)]
print(overdose_meds.shape)
display(overdose_meds.head(5))

15


,enc_date,enc_pat_id,enc_code,enc_description,enc_reason_code,enc_reason_description
enc_id,,,,,,
85134770-5582-49e5-983f-90b031b220f7,2008-03-27,3ff78458-b80e-4c45-9c6f-9417e8f103d6,50849002,Emergency Room Admission,55680006,Drug overdose
0dde55cc-668b-4f10-8962-93a26afc845b,2008-05-22,3ff78458-b80e-4c45-9c6f-9417e8f103d6,266707007,Drug addiction therapy,NaN,NaN
7a6b6733-2435-4c1b-8925-c30a5ea10384,2008-06-21,3ff78458-b80e-4c45-9c6f-9417e8f103d6,266707007,Drug addiction therapy,NaN,NaN
ee4c57d8-20d7-4085-ba0a-d068a3a2d73c,2008-07-03,3ff78458-b80e-4c45-9c6f-9417e8f103d6,185349003,Encounter for check up (procedure),NaN,NaN
8d3a7ff5-8881-4d3f-b7c2-4d2c1393e52b,2008-07-21,3ff78458-b80e-4c45-9c6f-9417e8f103d6,266707007,Drug addiction therapy,NaN,NaN


(41, 8)


,med_start_date,med_stop_date,med_pat_id,med_enc_id,med_code,med_description,med_reason_code,med_reason_description
61,2010-03-25,2010-08-19,3ff78458-b80e-4c45-9c6f-9417e8f103d6,fffd3a0d-090a-4c2d-b369-86d904067dfa,106892,insulin human isophane 70 UNT/ML / Regular In...,44054006,Diabetes
62,2010-08-19,NaN,3ff78458-b80e-4c45-9c6f-9417e8f103d6,05023946-5b76-442f-8497-5728ee69146d,865098,Insulin Lispro 100 UNT/ML Injectable Solution ...,44054006,Diabetes
63,2011-02-28,2011-03-07,3ff78458-b80e-4c45-9c6f-9417e8f103d6,7683553c-873f-4880-9926-7dc2118088fd,608680,Acetaminophen 160 MG,10509002,Acute bronchitis (disorder)
64,2015-01-29,NaN,3ff78458-b80e-4c45-9c6f-9417e8f103d6,cd60dc1c-6de8-402a-a72e-39a5a7b76eee,309362,Clopidogrel 75 MG Oral Tablet,NaN,NaN
65,2015-01-29,NaN,3ff78458-b80e-4c45-9c6f-9417e8f103d6,cd60dc1c-6de8-402a-a72e-39a5a7b76eee,564666,Nitroglycerin 0.4 MG/ACTUAT [Nitrolingual],NaN,NaN


In [40]:
# Display all the Opioid medications
opioids_rxnorm = ['1049369','1310197','1049544']
op1 = medications[medications.med_code.isin(opioids_rxnorm)]
print(op1.shape)
display(op1.head(10))

(7, 8)


,med_start_date,med_stop_date,med_pat_id,med_enc_id,med_code,med_description,med_reason_code,med_reason_description
131,2010-06-08,2010-07-27,4154eae6-393c-483d-8678-3a8efdf5b5cf,68ebde61-5532-4a1c-bed3-ce865e0211bd,1049544,oxyCODONE Hydrochloride 15 MG [OxyCONTIN],NaN,NaN
531,2015-06-17,2015-06-24,ca51b240-5e56-4bf7-9420-fae4cebf10f0,3935b106-e7da-405c-ab00-7e2d73570dad,1310197,Acetaminophen 300 MG / HYDROcodone Bitartrate ...,NaN,NaN
1100,2016-11-09,2016-11-16,019d9fa6-9481-408c-b4af-620530033f21,bc3af043-63a1-46ba-a102-6467fb6479bd,1310197,Acetaminophen 300 MG / HYDROcodone Bitartrate ...,NaN,NaN
1330,2015-09-25,2015-10-09,3540ba69-ffa2-4fbf-8de9-2ec21dbff0d6,dd0c6e2d-da56-429c-ae36-9834dc9e1ae6,1310197,Acetaminophen 300 MG / HYDROcodone Bitartrate ...,NaN,NaN
2229,2012-07-04,2012-07-11,f6ffe77f-8451-4ad9-a37d-01bdc56b8f42,a3413663-e972-4fa3-8929-edb499902b52,1310197,Acetaminophen 300 MG / HYDROcodone Bitartrate ...,NaN,NaN
2384,2013-07-04,2013-08-15,22006114-03b3-439d-bf6a-876be4b66e64,7a96a88c-15c5-4936-936d-604e0151dfb3,1310197,Acetaminophen 300 MG / HYDROcodone Bitartrate ...,NaN,NaN
2928,2008-06-24,2008-07-01,3587b86b-ee6a-44e3-83f1-f801f5893d5f,0633fdd7-56f7-4ce3-92f8-cdd0cc486ab4,1049544,oxyCODONE Hydrochloride 15 MG [OxyCONTIN],NaN,NaN


In [31]:
# Pickle the data
datadir = '/home/jovyan/work/dswg/data'
for name, dt in data.items():
    print('Pickling {} of shape: {}'.format(name, dt.shape))
    dt.to_pickle('{}/temp/{}.tmp'.format(datadir, name))

Pickling patients of shape: (1118, 16)
Pickling encounters of shape: (26509, 6)
Pickling observations of shape: (112205, 7)
Pickling medications of shape: (4627, 8)
